In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
car=pd.read_csv('quikr_car.csv')
car.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [3]:
car.shape

(892, 6)

In [4]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


### Cleaning

In [5]:
backup=car.copy()# Backup of Dataset

### Column - Year

In [6]:
car=car[car['year'].str.isnumeric()]
car.shape

(842, 6)

In [7]:
car['year']=car['year'].astype('int32')
car['year'].dtypes

dtype('int32')

### Column- Price


In [8]:
car=car[car['Price']!="Ask For Price"]
car.shape

(819, 6)

In [9]:
## Replace , with blank
car['Price']=car['Price'].str.replace(',','').astype(int)
car['Price'].dtypes

dtype('int32')

### Column - Kms Driven


In [10]:
car['kms_driven']=car['kms_driven'].str.split(' ').str.get(0).str.replace(',','')

In [11]:
car=car[car['kms_driven'].str.isnumeric()]
car['kms_driven']=car['kms_driven'].astype(int)

In [12]:
car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 817 entries, 0 to 889
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        817 non-null    object
 1   company     817 non-null    object
 2   year        817 non-null    int32 
 3   Price       817 non-null    int32 
 4   kms_driven  817 non-null    int32 
 5   fuel_type   816 non-null    object
dtypes: int32(3), object(3)
memory usage: 35.1+ KB


### Column - FuelType

In [13]:
car=car[~car['fuel_type'].isna()]

### Column-Name

In [14]:
car['name']=car['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [15]:
car.reset_index(drop=True)

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
811,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
812,Tata Indica V2,Tata,2009,110000,30000,Diesel
813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
814,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [16]:
car.describe()

,year,Price,kms_driven
count,816.000000,8.160000e+02,816.000000
mean,2012.444853,4.117176e+05,46275.531863
std,4.002992,4.751844e+05,34297.428044
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.912500e+05,56818.500000
max,2019.000000,8.500003e+06,400000.000000


In [17]:
car=car[car['Price']<6e6].reset_index(drop=True)
car

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
810,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
811,Tata Indica V2,Tata,2009,110000,30000,Diesel
812,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
813,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [18]:
car.to_csv('Cleaned data')

In [20]:
import numpy as np
X=car.drop(columns='Price')
y=np.log(car['Price'])

##  Linear Regression Model

In [21]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.metrics import r2_score , mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=np.argmax(scores))

ValueError: attempt to get argmax of an empty sequence

In [32]:
scores=[]
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))
    

NameError: name 'column_trans' is not defined

In [33]:
ohe=OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])


OneHotEncoder()

In [34]:
step1=make_column_transformer((OneHotEncoder(categories=ohe.categories_),
                                      ['name','company','fuel_type']),
                                     remainder='passthrough')
step2=LinearRegression()
pipe=make_pipeline(step1,step2)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.7562154419512663
0.24190232765268646


### Lasso Regression

In [35]:
# step1=make_column_transformer((OneHotEncoder(categories=ohe.categories_),
#                                       ['name','company','fuel_type']),
#                                      remainder='passthrough')
# step2=Lasso(alpha=0.001)
# pipe=make_pipeline(step1,step2)
# pipe.fit(X_train,y_train)
# y_pred=pipe.predict(X_test)
# print(r2_score(y_test,y_pred))
# print(mean_absolute_error(y_test,y_pred))

### Ridge Regression

In [36]:
# step1=make_column_transformer((OneHotEncoder(categories=ohe.categories_),
#                                       ['name','company','fuel_type']),
#                                      remainder='passthrough')
# step2=Ridge(alpha=0.01)
# pipe=make_pipeline(step1,step2)
# pipe.fit(X_train,y_train)
# y_pred=pipe.predict(X_test)
# print(r2_score(y_test,y_pred))
# print(mean_absolute_error(y_test,y_pred))

###  Decision Tree

In [37]:
# from sklearn.tree import DecisionTreeRegressor
# step1=make_column_transformer((OneHotEncoder(categories=ohe.categories_),
#                                       ['name','company','fuel_type']),
#                                      remainder='passthrough')
# step2=DecisionTreeRegressor(max_depth=8)
# pipe=make_pipeline(step1,step2)
# pipe.fit(X_train,y_train)
# y_pred=pipe.predict(X_test)
# print(r2_score(y_test,y_pred))
# print(mean_absolute_error(y_test,y_pred))

### Random Forest

In [38]:
# scores=[]
# for i in range(1000):
#     X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=441)
#     step2=RandomForestRegressor(n_estimators=200,
#                              random_state=i,
#                              max_samples=0.5,
#                              max_features=0.75,
#                              max_depth=15)

#     pipe=make_pipeline(column_trans,lr)
#     pipe.fit(X_train,y_train)
#     y_pred=pipe.predict(X_test)
#     scores.append(r2_score(y_test,y_pred))
# print(np.argmax(scores))

In [39]:
# from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor

# step1=make_column_transformer((OneHotEncoder(categories=ohe.categories_),
#                                       ['name','company','fuel_type']),
#                                      remainder='passthrough')

# step2=RandomForestRegressor(n_estimators=100,
#                              random_state=0,
#                              max_samples=0.5,
#                              max_features=0.75,
#                              max_depth=15)

# pipe=make_pipeline(step1,step2)
# pipe.fit(X_train,y_train)
# y_pred=pipe.predict(X_test)
# print(r2_score(y_test,y_pred))
# print(mean_absolute_error(y_test,y_pred))

In [40]:
# from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor

# step1=make_column_transformer((OneHotEncoder(categories=ohe.categories_),
#                                       ['name','company','fuel_type']),
#                                      remainder='passthrough')

# step2=AdaBoostRegressor(estimator=DecisionTreeRegressor(),
#                         random_state=42,
#                         n_estimators=100,
#                         learning_rate=1
#                        )

# pipe=make_pipeline(step1,step2)
# pipe.fit(X_train,y_train)
# y_pred=pipe.predict(X_test)
# print(r2_score(y_test,y_pred))
# print(mean_absolute_error(y_test,y_pred))

In [41]:
import pickle
pickle.dump(pipe,open('model.pkl','wb'))

In [42]:
np.exp(pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']],columns=['name','company','year','kms_driven','fuel_type'])))

array([458356.22814363])